In [1]:
import pandas as pd
import json
import sys
import yfinance
from googlemaps import Client as GoogleMaps
sys.path.append('../src')
sys.path.append('../src/data')
sys.path.append('..')
from api_config import GOOGLE_API_KEY
#from smartsheet_data import SAMPLE_SHEET_ID, PROJECT_SHEET_ID, SUPPLY_CHAIN_SHEET_ID, CFA_TRACKER_SHEET_ID, get_smartsheet_data, sheet_to_dataframe

In [4]:
import sys
import pandas as pd
sys.path.append('../data')
sys.path.append('..')
from data.projects import sync_projects
from app import db

In [6]:
from data.projects import clean_cfa_tracker_df, clean_sc_df, sheet_to_dataframe, sync_projects, get_smartsheet_data, SUPPLY_CHAIN_SHEET_ID, CFA_TRACKER_SHEET_ID

In [58]:
cfa_tracker_df_raw

,Location_Number,Region,DM,CFA_CM,Prototype,TIER,Area,CD_Release_Date,Permit_Set_Due,Bid_Set_Due,...,Hood_wall___full_height/infill,BIM360_Access,Frog_Proj_Number:_SEOR,Frog_Proj_Number:_PNL_Design,S+S_Geo_Rpt_Reviewed,Proposal_Sent,Proposal_approved_+_logged,Intake_Completed,Frog_Scope,Were_Shops_Generated?
0,563.0,Southwest Region,None,None,100.0,None,None,2000-02-01,None,None,...,None,None,None,None,None,None,None,False,None,None
1,4055.0,Northeast Region,John Martinez,None,P12 LS-L-R,TIER 1,5108.0,2018-08-10,None,None,...,Full height,None,None,None,None,None,None,False,"Prefab Shops, Prefab Components",None
2,805.0,Southeast Region,Joseph Latimer,None,90B,None,None,None,None,None,...,None,None,None,None,None,None,None,False,None,None
3,4066.0,Southwest Region,Steve Lewis,Ray Parrish,P12 LS-L-R,None,4911.0,2017-11-02,None,None,...,Full height,None,None,None,None,None,None,False,"Prefab Shops, Prefab Components",None
4,4083.0,Atlantic Region,Nate Thompson,None,P12 LS-L-R,None,None,2018-02-05,None,None,...,None,None,None,None,None,None,None,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,4463.0,West Region,None,None,P12 SE-L,None,4863.0,None,None,None,...,None,None,None,None,None,None,None,False,None,None
435,4594.0,Northeast Region,None,None,P13 LS-L,None,5000.0,2020-11-20,None,None,...,None,None,None,None,None,None,None,False,None,None
436,00825,Southeast,#NO MATCH,None,P13 SE-L,TIER 1,None,None,None,None,...,None,None,None,None,None,None,None,False,None,None
437,04945,Midwest,None,None,P13 LSR-L,TIER 2,None,None,None,None,...,None,None,CHIC0498,CHIC0499,None,None,None,False,None,None


## STEP 1 - Clean Supply Chain DF

In [97]:

# get columns
cols = ['Location_Number','Prototype', 'Region','Awarded_Wall_Panel_Supplier','SO_Amount','SO_Executed_Date','Panels_On_Site','sqft','Component_Model_Status','Wall_Panels','#_of_Ext_Wall_panel','#_of_Int_Wall_panel','sqft_of_Int_Wall_panel','sqft_of_Ext_Wall_panel']
# filter out any project that does not have a wall panels cost
sc_df=sc_df_raw[sc_df_raw['Wall_Panels'].notnull()][cols]
# drop extra NA values (usually at the end)
sc_df.dropna(how="all",inplace=True, axis=1)
sc_df['#_of_Ext_Wall_panel'] = sc_df[sc_df.apply(pd.to_numeric, errors='coerce').notna()].dropna()
# force location number to type (removes leading zeroes)
sc_df = sc_df.astype({'Location_Number':'int32'})
# fill empty numerical cells with zeroes
sc_df.fillna({'#_of_Int_Wall_panel':0,'#_of_Ext_Wall_panel':0,'sqft_of_Int_Wall_panel':0})
sc_df



,Location_Number,Prototype,Region,Awarded_Wall_Panel_Supplier,SO_Executed_Date,Panels_On_Site,sqft,Component_Model_Status,Wall_Panels,#_of_Ext_Wall_panel,#_of_Int_Wall_panel,sqft_of_Int_Wall_panel,sqft_of_Ext_Wall_panel
0,4055,P12 LS-L-R,Northeast Region,Mitsui,2018-06-18,2018-08-10,4880.0,#NO MATCH,91953.0,64.0,49.0,3199.0,6576.0
1,4066,P12 LS-L-R,Southwest Region,Mitsui,2018-08-09,2018-10-08,4880.0,#NO MATCH,96127.0,80.0,46.0,3013.0,7336.0
2,4022,P12 LS-M,Southwest Region,Mitsui,2018-10-01,2018-10-08,4880.0,#NO MATCH,90525.0,56.0,46.0,3157.0,5298.0
3,4048,P12 LC-L,West Region,Golden State,2019-01-04,2019-02-11,4844.0,#NO MATCH,80206.0,68.0,46.0,3401.0,5303.0
4,643,P12 SE-L,Southeast Region,Golden State,2019-02-22,2019-04-15,4859.0,#NO MATCH,91510.0,57.0,57.0,4109.0,5757.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,4435,P13 SE-L,Northeast Region,RedBuilt,2021-08-10,2021-11-04,5147.0,8. FABRICATED,127500.0,58.0,0.0,NaN,5618.0
85,832,P12 LS-L-R,Southeast Region,RedBuilt,2020-02-24,2021-11-08,4996.0,7. RELEASED TO PRODUCTION,73000.0,44.0,27.0,2299.0,5287.0
86,4765,P13 SE-M,Northeast Region,RedBuilt,2021-08-10,2021-12-02,4592.0,7. RELEASED TO PRODUCTION,126280.0,56.0,0.0,NaN,5364.0
99,4577,P13-LSR-L,Northeast Region,RedBuilt,2021-07-06,2022-04-28,4998.0,7. RELEASED TO PRODUCTION,148880.0,54.0,0.0,NaN,6179.0


## STEP 2 - Clean CFA DF

In [132]:
cols = ["Location_Number","Address", "City", "State"]
cfa_tracker_df = cfa_tracker_df_raw[cols]
cfa_tracker_df["Location_Number"] = pd.to_numeric(cfa_tracker_df["Location_Number"],downcast='integer',errors='coerce').fillna(0)
cfa_tracker_df = cfa_tracker_df.astype({'Location_Number':'int32'})
cfa_tracker_df.dropna(axis=0,how="any",inplace=True)
cfa_tracker_df

C:\Users\jbuccola\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Location_Number,Address,City,State
0,563,1300 E Jackson Ave,Mcallen,TX
1,4055,12101 Winchester Rd,Lavale,MD
2,805,604 Holcomb Bridge Rd,Roswell,GA
3,4066,15500 Hwy 35,Buda,TX
4,4083,6885 Getwell Rd N,Southaven,MS
...,...,...,...,...
433,4342,3115 Chestnut St # 3133,Camp Hill,PA
434,4463,1600 W Armory Way,Seattle,OR
435,4594,Warwick Township,Warwick Township,PA
436,825,3066 Washington Rd,Augusta,GA


## STEP 3 - Merge Dataframes

In [133]:
merged = sc_df.merge(cfa_tracker_df,how='left',left_on='Location_Number',right_on='Location_Number')
merged


,Location_Number,Prototype,Region,Awarded_Wall_Panel_Supplier,SO_Executed_Date,Panels_On_Site,sqft,Component_Model_Status,Wall_Panels,#_of_Ext_Wall_panel,#_of_Int_Wall_panel,sqft_of_Int_Wall_panel,sqft_of_Ext_Wall_panel,Address,City,State
0,4055,P12 LS-L-R,Northeast Region,Mitsui,2018-06-18,2018-08-10,4880.0,#NO MATCH,91953.0,64.0,49.0,3199.0,6576.0,12101 Winchester Rd,Lavale,MD
1,4066,P12 LS-L-R,Southwest Region,Mitsui,2018-08-09,2018-10-08,4880.0,#NO MATCH,96127.0,80.0,46.0,3013.0,7336.0,15500 Hwy 35,Buda,TX
2,4022,P12 LS-M,Southwest Region,Mitsui,2018-10-01,2018-10-08,4880.0,#NO MATCH,90525.0,56.0,46.0,3157.0,5298.0,3442 Palmer Hwy,Texas City,TX
3,4048,P12 LC-L,West Region,Golden State,2019-01-04,2019-02-11,4844.0,#NO MATCH,80206.0,68.0,46.0,3401.0,5303.0,2940 SW Cedar Hills Blvd,Beaverton,OR
4,643,P12 SE-L,Southeast Region,Golden State,2019-02-22,2019-04-15,4859.0,#NO MATCH,91510.0,57.0,57.0,4109.0,5757.0,352 Northside Dr E,Statesboro,GA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,4435,P13 SE-L,Northeast Region,RedBuilt,2021-08-10,2021-11-04,5147.0,8. FABRICATED,127500.0,58.0,0.0,NaN,5618.0,70 NJ Route 36,Eatontown,NJ
82,832,P12 LS-L-R,Southeast Region,RedBuilt,2020-02-24,2021-11-08,4996.0,7. RELEASED TO PRODUCTION,73000.0,44.0,27.0,2299.0,5287.0,211 Lakeshore Pkwy,Homewood,AL
83,4765,P13 SE-M,Northeast Region,RedBuilt,2021-08-10,2021-12-02,4592.0,7. RELEASED TO PRODUCTION,126280.0,56.0,0.0,NaN,5364.0,374 William S. Canning Boulevard,Fall River,MA
84,4577,P13-LSR-L,Northeast Region,RedBuilt,2021-07-06,2022-04-28,4998.0,7. RELEASED TO PRODUCTION,148880.0,54.0,0.0,NaN,6179.0,124 East Jericho Turnpike,Huntingtn Sta,NY


In [ ]:
cleaned_df = clean_projects_dataframe(combined_df)

In [7]:
cfa_tracker_df_raw = sheet_to_dataframe(get_smartsheet_data(CFA_TRACKER_SHEET_ID)[1])

Starting ...
Loaded 439 rows from sheet: Panelization Project Tracking
Done


In [8]:
cfa_clean_df = clean_cfa_tracker_df(cfa_tracker_df_raw)
cfa_clean_df

..\data\projects.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfa_tracker_df["Location_Number"] = pd.to_numeric(cfa_tracker_df["Location_Number"],downcast='integer',errors='coerce').fillna(0)


,Location_Number,Address,City,State
0,563,1300 E Jackson Ave,Mcallen,TX
1,4055,12101 Winchester Rd,Lavale,MD
2,805,604 Holcomb Bridge Rd,Roswell,GA
3,4066,15500 Hwy 35,Buda,TX
4,4083,6885 Getwell Rd N,Southaven,MS
...,...,...,...,...
433,4342,3115 Chestnut St # 3133,Camp Hill,PA
434,4463,1600 W Armory Way,Seattle,OR
435,4594,Warwick Township,Warwick Township,PA
436,825,3066 Washington Rd,Augusta,GA


In [11]:
cfa_clean_df[cfa_clean_df['Location_Number'] == 813]

,Location_Number,Address,City,State
